___

# An example of gathering AHS data for HPE iLO systems

___

___

In [1]:
import sys
import json

_importing python modules_

In [2]:
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

ModuleNotFoundError: No module named 'redfish'

<div class="alert alert-block alert-info">
<b>---:</b> using refish libraries
</div>

In [3]:
def get_ahs_data(_redfishobj, logfile):

    active_health_system_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        active_health_system_uri = managers_members_response.obj.Oem.Hpe.Links\
                                    ['ActiveHealthSystem']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeiLOActiveHealthSystem.' in instance['@odata.type']:
                active_health_system_uri = instance['@odata.id']

    if active_health_system_uri:
        active_health_system_response = _redfishobj.get(active_health_system_uri)
        active_health_system_log_uri = active_health_system_response.obj.Links['AHSLocation']\
                                                                                        ['extref']
        active_health_system_log_resp = _redfishobj.get(active_health_system_log_uri)
        if active_health_system_log_resp.status == 400:
            try:
                print(json.dumps(active_health_system_log_resp.obj['error']\
                                 ['@Message.ExtendedInfo'], indent=4, sort_keys=True))
            except Exception:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif active_health_system_log_resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            with open(logfile, 'wb') as ahsoutput:
                ahsoutput.write(active_health_system_log_resp.ori)
                ahsoutput.close()
                sys.stdout.write("AHS Data saved successfully as: \'%s\'" % logfile)




<div class="alert alert-block alert-warning">
<b>if condition:</b> if we do not have a resource directory or want to force it's non use to find the relevant URI<br>
    <b>else:</b>Use Resource directory to find the relevant URI
</div>

<div class="alert alert-block alert-info">
<b>When running on the server locally use the following commented values</b> 
    SYSTEM_URL = None<br>
    LOGIN_ACCOUNT = None<br>
    LOGIN_PASSWORD = None<br>
</div>
    

<div class="alert alert-block alert-info">
<b>When running remotely</b> 
    connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:<br>
    # "https://10.0.0.100"<br>
    # "https://ilo.hostname"<br>
</div>


In [4]:
if __name__ == "__main__":
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    # logfile path and filename
    LOGFILE = "data.ahs"
   
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    get_ahs_data(REDFISHOBJ, LOGFILE)
    REDFISHOBJ.logout()


NameError: name 'ServerDownOrUnreachableError' is not defined

<div class="alert alert-block alert-danger">
<b>Just don't:</b>
     flag to force disable resource directory. Resource directory and associated operations are  intended for HPE servers.
</div>